In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import graphviz

In [5]:
with open('input.txt') as f:
    data = f.read().splitlines()
data

['S â†’ AB', 'A â†’ a', 'B â†’ C | b', 'C â†’ D', 'D â†’ E', 'E â†’ a']

In [6]:
CFG = {}
for i in data:
    print(i)
    var = i[0]
    j = i.split(' | ')
    t = 1
    CFG[var] = []
    CFG[var].append(j[0].split(" â†’ ")[1])
    if len(j) > 1:
        while (t < len(j)):
            CFG[var].append(j[t])
            t += 1
print(CFG)

S â†’ AB
A â†’ a
B â†’ C | b
C â†’ D
D â†’ E
E â†’ a
{'S': ['AB'], 'A': ['a'], 'B': ['C', 'b'], 'C': ['D'], 'D': ['E'], 'E': ['a']}


# Eliminate Unit Productions

In [7]:
#1st finding out all states that have single terminal in RHS
single_terminals = []
for i in CFG:
    if len(CFG[i])==1 and CFG[i][0].islower():
        single_terminals.append(i)
print(single_terminals)

['A', 'E']


In [8]:
# Now, checking if any of the single terminals are in RHS of any other state
while True:
    changed = False
    for i in CFG:
        for j in CFG[i]:
            for l in j:
                if l in single_terminals:
                    k = single_terminals.index(l)
                    CFG[i].append(j.replace(single_terminals[k], CFG[single_terminals[k]][0]))
                    CFG[i].remove(j)
                    if len(CFG[i])==1:
                        single_terminals.append(i)
                    print(single_terminals)
                    changed = True
    if not changed:
        break
print(CFG)

['A', 'E', 'S']
['A', 'E', 'S', 'D']
['A', 'E', 'S', 'D', 'C']
['A', 'E', 'S', 'D', 'C']
{'S': ['aB'], 'A': ['a'], 'B': ['b', 'a'], 'C': ['a'], 'D': ['a'], 'E': ['a']}


In [9]:
# Now, checking if any of the single terminals are in RHS of any other state
for i in CFG:
    for j in CFG[i]:
        for k in single_terminals:
            if k in j:
                #single_terminals.append(k)
                CFG[i].append(j.replace(k, CFG[k][0]))
                CFG[i].remove(j)
print(CFG)

{'S': ['aB'], 'A': ['a'], 'B': ['b', 'a'], 'C': ['a'], 'D': ['a'], 'E': ['a']}


# Elimate Useless Productions

# 1) Unreachable

In [10]:
reachable_states = []
given_states = []
terminals = []
for i in CFG:
    if i not in given_states:
        given_states.append(i)
    for j in CFG[i]:
        for l in j:
            if l not in reachable_states:
                if l.isupper():
                    reachable_states.append(l)
                else:
                    terminals.append(l)
print(reachable_states)
print(given_states)
print(terminals)

['B']
['S', 'A', 'B', 'C', 'D', 'E']
['a', 'a', 'b', 'a', 'a', 'a', 'a']


In [11]:
unreachable = []
for i in given_states:
    if i=="S":
        continue
    elif i not in reachable_states:
        unreachable.append(i)
print(unreachable)

['A', 'C', 'D', 'E']


In [12]:
for i in unreachable:
    CFG.pop(i)
print(CFG)

{'S': ['aB'], 'B': ['b', 'a']}


# Eliminate Null Productions

# Convert to Chomsky Normal Form

In [ ]:
def cfg_to_cnf(cfg):
    # Step 1: Add new start symbol if necessary
    if 'S' in cfg and any('S' in rhs for rhs in cfg.values()):
        new_start = 'S\''
        while new_start in cfg:
            new_start += '\''
        cfg[new_start] = ['S']
        del cfg['S']
        for symbol, rhs in cfg.items():
            cfg[symbol] = [new_start if s == 'S' else s for s in rhs]
    
    # Step 2: Remove null productions (not needed)
    
    # Step 3: Remove unit productions (not needed)
    
    # Step 4: Replace A → aB with A → XB, X → a
    new_rules = {}
    for symbol, rhs in cfg.items():
        new_rhs = []
        for prod in rhs:
            if len(prod) == 1:
                new_rhs.append(prod)
            else:
                for i, s in enumerate(prod):
                    if s.islower():
                        new_nonterm = f'{symbol}_{i}'
                        if new_nonterm not in new_rules:
                            new_rules[new_nonterm] = [s]
                        new_rhs.append(new_nonterm)
                        break
                    elif i == len(prod) - 1:
                        new_rhs.append(prod)
        cfg[symbol] = new_rhs
    for symbol, rhs in new_rules.items():
        cfg[symbol] = rhs
    
    # Step 3: Replace A → B1B2...Bn with A → B1C, C → B2...Bn
    new_rules = {}
    for symbol, rhs in cfg.items():
        new_rhs = []
        for prod in rhs:
            if len(prod) <= 2:
                new_rhs.append(prod)
            else:
                new_nonterm = f'{symbol}_1'
                if new_nonterm not in new_rules:
                    new_rules[new_nonterm] = prod[1:]
                new_rhs.append(prod[:2] + [new_nonterm])
                for i in range(2, len(prod)):
                    prev_nonterm, new_nonterm = new_nonterm, f'{symbol}_{i}'
                    if new_nonterm not in new_rules:
                        new_rules[new_nonterm] = prod[i:]
                    new_rhs.append([prev_nonterm, new_nonterm])
        cfg[symbol] = new_rhs
    for symbol, rhs in new_rules.items():
        cfg[symbol] = [rhs]
    
    return cfg
CFG = {'S': ['AB'], 'A': ['a'], 'B': ['C', 'b'], 'C': ['D'], 'D': ['E'], 'E': ['a']}
cfg_to_cnf(CFG)

: 

In [45]:
def eliminate_mixed_terms(cfg):
    cnf = {}
    variables = set(cfg.keys())
    
    # Step 1: Replace mixed terminals and non-terminals with new variables
    for variable, productions in cfg.items():
        new_productions = []
        for production in productions:
            new_production = ''
            for symbol in production:
                if symbol.islower():
                    new_var = symbol.upper()
                    while new_var in variables:
                        new_var += "'"
                    variables.add(new_var)
                    cnf[new_var] = [symbol]
                    new_production += new_var
                else:
                    new_production += symbol
            new_productions.append(new_production)
        cnf[variable] = new_productions
    
    # Step 2: Eliminate non-unit productions
    for variable, productions in cnf.items():
        new_productions = []
        for production in productions:
            if len(production) > 1:
                new_production = ''
                for symbol in production:
                    new_production += symbol + "'"
                    while new_production in variables:
                        new_production += "'"
                    variables.add(new_production)
                    cnf[new_production] = [symbol]
                new_productions.append(new_production)
            else:
                new_productions.append(production)
        cnf[variable] = new_productions
    
    return cnf


In [46]:
cfg = {'S': ['aSb', 'ab']}
cnf = eliminate_mixed_terms(cfg)
print(cnf)


RuntimeError: dictionary changed size during iteration

In [21]:
cfg = {
    'S': ['aAB', 'bBA'],
    'A': ['a', 'aS', 'bAA'],
    'B': ['b', 'bS', 'aBB']
}

cnf = convert_to_cnf(cfg)

# for lhs, rhs_list in cnf.items():
#     for rhs in rhs_list:
#         print(f'{lhs} -> {rhs}')

KeyError: 'non_terminals'

# Plot the finite Automata

In [ ]:
import graphviz

def plot_automaton(grammar):
    """Plot the finite automaton corresponding to a CFG."""
    # Create a new graph
    graph = graphviz.Digraph(format='png')

    # Add states and transitions for each production in the CFG
    for var, prods in grammar.items():
        # Add the state for the variable
        graph.node(var)
        for prod in prods:
            # If the production is a single terminal symbol, add a transition to a new dummy state
            if len(prod) == 1 and prod[0].islower():
                graph.edge(var, 'D0', label=prod[0])
            else:
                # Add a transition for each symbol in the production
                for i in range(len(prod)):
                    graph.edge(var, prod[i])
                    # If the symbol is a nonterminal that produces a single terminal symbol, add a transition to a dummy state
                    if len(prod[i]) == 1 and prod[i].islower():
                        graph.edge(prod[i], 'D0', label=prod[i])

    # Add a dummy state for single-terminal productions
    graph.node('D0', shape='diamond')

    # Set the graph attributes
    graph.attr(rankdir='LR', size='8,5')

    # Render and display the graph
    graph.render(directory="C:\\Users\\DEVANSH\\Downloads\\THOC_Innovative", view=True)  
    print(graph)

In [ ]:
cfg = {'S': ['AC'], 'A': ['a'], 'C': ['c', 'BC']}
plot_automaton(cfg)